## Tp2 ift 870
Quentin Levieux<br>
Frederic Giasson

In [50]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import numbers

In [51]:
package = pd.read_csv("Package2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
display("Dimension de package.csv : ",package.shape)
product = pd.read_csv("Product2.csv",sep=";", error_bad_lines=False, encoding ='latin1')
print("Dimension de product.csv: ",product.shape)
print("Nombre d'attributs de package:", len(package.columns))
print("Nombre d'attributs de product:", len(product.columns))
print("\nAttributs de package: \n",package.columns.values)
print("\nAttributs de product: \n",product.columns.values)
commonAttributs = list(set(package).intersection(product))
print("\nAttributs communs au deux tables :\n",commonAttributs)

'Dimension de package.csv : '

(173887, 8)

Dimension de product.csv:  (93238, 20)
Nombre d'attributs de package: 8
Nombre d'attributs de product: 20

Attributs de package: 
 ['PRODUCTID' 'PRODUCTNDC' 'NDCPACKAGECODE' 'PACKAGEDESCRIPTION'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'NDC_EXCLUDE_FLAG'
 'SAMPLE_PACKAGE']

Attributs de product: 
 ['PRODUCTID' 'PRODUCTNDC' 'PRODUCTTYPENAME' 'PROPRIETARYNAME'
 'PROPRIETARYNAMESUFFIX' 'NONPROPRIETARYNAME' 'DOSAGEFORMNAME' 'ROUTENAME'
 'STARTMARKETINGDATE' 'ENDMARKETINGDATE' 'MARKETINGCATEGORYNAME'
 'APPLICATIONNUMBER' 'LABELERNAME' 'SUBSTANCENAME'
 'ACTIVE_NUMERATOR_STRENGTH' 'ACTIVE_INGRED_UNIT' 'PHARM_CLASSES'
 'DEASCHEDULE' 'NDC_EXCLUDE_FLAG' 'LISTING_RECORD_CERTIFIED_THROUGH']

Attributs communs au deux tables :
 ['STARTMARKETINGDATE', 'NDC_EXCLUDE_FLAG', 'ENDMARKETINGDATE', 'PRODUCTNDC', 'PRODUCTID']


In [52]:
product.head(10)

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,NaN,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710,NaN,NDA,NDA018781,10,WATER,1,mL/mL,NaN,NaN,N,20201231.0
1,NaN,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601,NaN,NDA,NDA202008,10,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20211231.0
2,NaN,0002-1433,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,.75,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
3,NaN,0002-1434,HUMAN PRESCRIPTION DRUG,Trulicity,NaN,Dulaglutide,"INJECTION, SOLUTION",SUBCUTANEOUS,20140918,NaN,BLA,BLA125469,10,DULAGLUTIDE,1.5,mg/.5mL,"GLP-1 Receptor Agonist [EPC],Glucagon-Like Pep...",NaN,N,20201231.0
4,NaN,0002-1436,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
5,NaN,0002-1445,HUMAN PRESCRIPTION DRUG,TALTZ,NaN,ixekizumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20160322,NaN,BLA,BLA125521,10,IXEKIZUMAB,80,mg/mL,"Interleukin-17A Antagonist [EPC],Interleukin-1...",NaN,N,20201231.0
6,NaN,0002-2377,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20180927,NaN,BLA,BLA761063,10,GALCANEZUMAB,120,mg/mL,NaN,NaN,N,20201231.0
7,NaN,0002-3115,HUMAN PRESCRIPTION DRUG,EMGALITY,NaN,galcanezumab,"INJECTION, SOLUTION",SUBCUTANEOUS,20190604,NaN,BLA,BLA761063,10,GALCANEZUMAB,100,mg/mL,NaN,NaN,N,20201231.0
8,NaN,0002-3227,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,10,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0
9,NaN,0002-3228,HUMAN PRESCRIPTION DRUG,Strattera,NaN,Atomoxetine hydrochloride,CAPSULE,ORAL,20021126,NaN,NDA,NDA021411,10,ATOMOXETINE HYDROCHLORIDE,25,mg/1,"Norepinephrine Reuptake Inhibitor [EPC],Norepi...",NaN,N,20211231.0


In [53]:
package.head(8)

,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710,NaN,N,N
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601,NaN,N,N
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200,0002-1200-50,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-50) > ...",20120601,NaN,N,N
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-61,2 SYRINGE in 1 CARTON (0002-1433-61) > .5 mL ...,20141107,NaN,N,Y
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433,0002-1433-80,4 SYRINGE in 1 CARTON (0002-1433-80) > .5 mL ...,20141107,NaN,N,N
5,0002-1434_42a80046-fd68-4b80-819c-a443b7816edb,0002-1434,0002-1434-61,2 SYRINGE in 1 CARTON (0002-1434-61) > .5 mL ...,20141107,NaN,N,Y
6,0002-1434_42a80046-fd68-4b80-819c-a443b7816edb,0002-1434,0002-1434-80,4 SYRINGE in 1 CARTON (0002-1434-80) > .5 mL ...,20141107,NaN,N,N
7,0002-1436_dd7f66eb-66e1-4ad0-a593-417dbf2c74e2,0002-1436,0002-1436-11,1 SYRINGE in 1 CARTON (0002-1436-11) > 1 mL i...,20180927,NaN,N,N


# Question 1

## Types des attributs

In [54]:
print("Types des attributs pour package :\n\n",package.dtypes)
print("\nTypes des attributs pour product :\n\n",product.dtypes)

Types des attributs pour package :

 PRODUCTID              object
PRODUCTNDC             object
NDCPACKAGECODE         object
PACKAGEDESCRIPTION     object
STARTMARKETINGDATE      int64
ENDMARKETINGDATE      float64
NDC_EXCLUDE_FLAG       object
SAMPLE_PACKAGE         object
dtype: object

Types des attributs pour product :

 PRODUCTID                            object
PRODUCTNDC                           object
PRODUCTTYPENAME                      object
PROPRIETARYNAME                      object
PROPRIETARYNAMESUFFIX                object
NONPROPRIETARYNAME                   object
DOSAGEFORMNAME                       object
ROUTENAME                            object
STARTMARKETINGDATE                    int64
ENDMARKETINGDATE                    float64
MARKETINGCATEGORYNAME                object
APPLICATIONNUMBER                    object
LABELERNAME                          object
SUBSTANCENAME                        object
ACTIVE_NUMERATOR_STRENGTH            object
ACTIVE_INGR

### Valeurs Manquantes

In [55]:
manquantePackage = package[package.isnull().any(axis=1)]
#display(manquantePackage)
print(manquantePackage.shape)

manquanteProduct = product[product.isnull().any(axis=1)]
#display(manquanteProduct)
print(manquanteProduct.shape)

print("Valeurs manquante par colonne dans package:\n")
for col in package.columns:
    nb = package[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans package :", len(package))

print("\nValeurs manquante par colonne dans product:\n")
for col in product.columns:
    nb = product[[col]].isna().sum()
    print(nb)
print("Nombre total de rangees dans product:", len(product))

(167507, 8)
(93238, 20)
Valeurs manquante par colonne dans package:

PRODUCTID    0
dtype: int64
PRODUCTNDC    1500
dtype: int64
NDCPACKAGECODE    2346
dtype: int64
PACKAGEDESCRIPTION    0
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    167431
dtype: int64
NDC_EXCLUDE_FLAG    0
dtype: int64
SAMPLE_PACKAGE    0
dtype: int64
Nombre total de rangees dans package : 173887

Valeurs manquante par colonne dans product:

PRODUCTID    1560
dtype: int64
PRODUCTNDC    0
dtype: int64
PRODUCTTYPENAME    0
dtype: int64
PROPRIETARYNAME    6
dtype: int64
PROPRIETARYNAMESUFFIX    83075
dtype: int64
NONPROPRIETARYNAME    4
dtype: int64
DOSAGEFORMNAME    0
dtype: int64
ROUTENAME    1932
dtype: int64
STARTMARKETINGDATE    0
dtype: int64
ENDMARKETINGDATE    88915
dtype: int64
MARKETINGCATEGORYNAME    0
dtype: int64
APPLICATIONNUMBER    13097
dtype: int64
LABELERNAME    0
dtype: int64
SUBSTANCENAME    2309
dtype: int64
ACTIVE_NUMERATOR_STRENGTH    2309
dtype: int64
ACTIVE_INGRED_UNIT  

### Incohérences dans package

In [118]:
#Verifier que PRODUCTNDC apparait au debut de PRODUCTID
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNull = package[(~package['PRODUCTNDC'].isnull()) & (~package['PRODUCTID'].isnull())]

arrayIndexIncoherentProductNDCAndProductIdForPackage = []
compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndProductIdForPackage.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence)    

Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC :  500


In [163]:
#Dans le cas ou le PRODUCTNDC et le NDCPackageCode existent
productNDCAndNDCPackageCodeNotNull = package[(~package['PRODUCTNDC'].isnull()) & (~package['NDCPACKAGECODE'].isnull())]

arrayIndexIncoherentProductNDCAndNDCPackageCode = []
compteurIncoherence = 0
for index, row in productNDCAndNDCPackageCodeNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    ndcpkgcode = row['NDCPACKAGECODE']
    if productndc != ndcpkgcode[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndNDCPackageCode.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans NDCPACKAGECODE différent du PRODUCTNDC : ", compteurIncoherence)

Nombre de PRODUCTNDC contenu dans NDCPACKAGECODE différent du PRODUCTNDC :  654


In [175]:
#Dans le cas ou le PRODUCTID et le NDCPackageCode existent
#Selon la documentation, le 8 premier caractere devraient etre identique dans PRODUCTID et NDCPACKAGECODE
#Car PRODUCTNDC = labeler code, product code. ProductID contient le PRODUCTNDC. NDCPACKAGECODE commence avec labeler code, product code
productIDAndNDCPackageCodeNotNull = package[(~package['PRODUCTID'].isnull()) & (~package['NDCPACKAGECODE'].isnull())]

arrayIndexIncoherentProductIDAndNDCPackageCode = []
compteurIncoherence = 0
for index, row in productIDAndNDCPackageCodeNotNull.iterrows() :
    productID = row['PRODUCTID'][:9]
    ndcpkgcode = row['NDCPACKAGECODE'][:9]
    if productID != ndcpkgcode[:len(productID)]:
        arrayIndexIncoherentProductIDAndNDCPackageCode.append(index)
        compteurIncoherence += 1
        
print("Nombre de NDCPACKAGECODE[:9] contenu dans PRODUCTID[:9] différent du NDCPACKAGECODE[:9] : ", compteurIncoherence)

Nombre de NDCPACKAGECODE[:9] contenu dans PRODUCTID[:9] différent du NDCPACKAGECODE[:9] :  154


In [209]:
#Aucune valeur manquante dans package pour la colonne STARTMARKETINGDATE donc on peut utiliser directement package
dateFormatNonValide = 0
arrayIndexInvalidStartMktgDateForPackage = []
for index, row in package.iterrows():
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(row['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        arrayIndexInvalidStartMktgDateForPackage.append(index)
        dateFormatNonValide +=1
        
print("Nombre de dates de STARTMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de STARTMARKETINGDATE invalides :  0


In [59]:
endMarketingDateNotnull = package[(~package['ENDMARKETINGDATE'].isnull())]

dateFormatNonValide = 0
for index in range(0,len(endMarketingDateNotnull)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(endMarketingDateNotnull.iloc[index]['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
        
print("Nombre de dates de ENDMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de ENDMARKETINGDATE invalides :  0


In [211]:
#Verifier que la date start soit avant ou égale à la date end (cependant date start peut être non valide)
#Une fois le start marketing date corrigee, il n'y a plus d'erreurs ici
datesIncoherentes = 0
arrayIndexDatesIncoherentesForPackage = []

for index, row in endMarketingDateNotnull.iterrows():
    dateEnd = pd.to_datetime(row['ENDMARKETINGDATE'])
    dateStart = pd.to_datetime(row['STARTMARKETINGDATE'])
    
    if dateEnd < dateStart :
        arrayIndexDatesIncoherentesForPackage.append(index)
        datesIncoherentes +=1
print("Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide : ",datesIncoherentes)

Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide :  0


In [61]:
#incoherence dans la table package
#SAMPLE_PACKAGE peut prendre une valeur parmis Y ou N
samplePackage = package[(package['SAMPLE_PACKAGE'] != 'Y') & (package['SAMPLE_PACKAGE'] != 'N')]

print("Nombre de ligne avec une valeur autre que Y ou N dans la colonne sample_Package : ",len(samplePackage))

#NDC_EXCLUDE_FLAG peut prendre une valeur parmis Y, N, E ou I
ndc_exclude_flag = package[(package['NDC_EXCLUDE_FLAG'] != 'Y') & (package['NDC_EXCLUDE_FLAG'] != 'N') & (package['NDC_EXCLUDE_FLAG'] != 'E') & (package['NDC_EXCLUDE_FLAG'] != 'I')]
print("Nombre de rangee avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag : ",len(ndc_exclude_flag))

Nombre de ligne avec une valeur autre que Y ou N dans la colonne sample_Package :  0
Nombre de rangee avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag :  0


En ce qui attrait au colonnes <b>PRODUCTID PRODUCTNDC NDCPACKAGECODE PACKAGEDESCRIPTION</b> de la table PACKAGE, il est difficile de d'avoir un critère qui permet de savoir si il y a une incoherence dans les données de la colonne. Pour ce qui est des autres colonnes, force est de constater qu'il n'y a pas d'incoherence flagrante dans les données intra colonnes. En effet, d'autant plus que les données de ces colonnes sont du type OBJECT, ils peuvent donc prendre n'importe quelle forme.

### Incohérences dans product

In [62]:
#Dans le cas ou le PRODUCTNDC et le PRODUCTID existent
productNDCAndProductIdNotNull = product[(~product['PRODUCTNDC'].isnull()) & (~product['PRODUCTID'].isnull())]

arrayIndexIncoherentProductNDCAndProductIdForProduct = []
compteurIncoherence = 0
for index, row in productNDCAndProductIdNotNull.iterrows() :
    productndc = row['PRODUCTNDC']
    productid = row['PRODUCTID']
    if productndc != productid[:len(productndc)]:
        arrayIndexIncoherentProductNDCAndProductIdForProduct.append(index)
        compteurIncoherence += 1
        
print("Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC : ", compteurIncoherence) 

Nombre de PRODUCTNDC contenu dans PRODUCTID différent du PRODUCTNDC :  513


In [63]:
#Dans le cas ou le MARKETINGCATEGORYNAME et le APPLICATIONNUMBER existent
#On ne teste pas sur MARKETINGCATEGORYNAME car il y a aucune valeur manquante
applicationNumberNotNull = product[(~product['APPLICATIONNUMBER'].isnull())]

compteurIncoherence = 0
for index in range(0,len(applicationNumberNotNull)) :
    marketingctgname = applicationNumberNotNull.iloc[index]['MARKETINGCATEGORYNAME']
    applicationnumber = applicationNumberNotNull.iloc[index]['APPLICATIONNUMBER']
    if marketingctgname != applicationnumber[:len(marketingctgname)]:
        compteurIncoherence += 1
        
print("Nombre de MARKETINGCATEGORYNAME contenu dans APPLICATIONNUMBER différent du MARKETINGCATEGORYNAME : ", compteurIncoherence)   

Nombre de MARKETINGCATEGORYNAME contenu dans APPLICATIONNUMBER différent du MARKETINGCATEGORYNAME :  31823


In [64]:
#NDC_EXCLUDE_FLAG peut prendre une valeur parmis Y, N, E ou I
ndc_exclude_flag = product[(product['NDC_EXCLUDE_FLAG'] != 'Y') & (product['NDC_EXCLUDE_FLAG'] != 'N') & (product['NDC_EXCLUDE_FLAG'] != 'E') & (product['NDC_EXCLUDE_FLAG'] != 'I')]
print("Nombre de ligne avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag : ",len(ndc_exclude_flag))

Nombre de ligne avec une valeur autre que Y, N, E ou I dans la colonne ndc_exclude_flag :  0


In [65]:
#MARKETINGCATEGORYNAME peut prendre une valeur parmis NDA, ANDA, BLA, OTC Monograph, ou Unapproved Drug
marketingctgname = product[(product['MARKETINGCATEGORYNAME'] != 'NDA') & (product['MARKETINGCATEGORYNAME'] != 'ANDA') & (product['MARKETINGCATEGORYNAME'] != 'BLA')]
print("Nombre de ligne avec une valeur autre que NDA, ANDA, BLA dans la colonne MARKETINGCATEGORYNAME : ",len(marketingctgname))

Nombre de ligne avec une valeur autre que NDA, ANDA, BLA dans la colonne MARKETINGCATEGORYNAME :  44782


In [66]:
for index in range (0, len(product['MARKETINGCATEGORYNAME'])):
    print(product['MARKETINGCATEGORYNAME'].iloc[index])

NDA
NDA
BLA
BLA
BLA
BLA
BLA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
BLA
NDA
NDA
NDA
BLA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
NDA
NDA
NDA
NDA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
BLA
BLA
BLA
BLA
NDA
NDA
BLA
BLA
NDA
NDA
NDA
NDA
NDA
BLA
NDA
BLA
BLA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
BLA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
BLA
OTC MONOGRAPH NOT FINAL
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
BLA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA
BLA


ANDA
ANDA
NDA
ANDA
OTC MONOGRAPH FINAL
ANDA
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
ANDA
ANDA
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA AUTHORIZED GENERIC
NDA AUTHORIZED GENERIC
NDA AUTHO

ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA
NDA
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
NDA
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA
NDA
NDA
ANDA
NDA
NDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
OTC MONOGRAPH FINAL
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
OTC MONOGRAPH NOT FINAL
ANDA
ANDA
ANDA
OTC MONOGRAPH NOT FINAL
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
AN

UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED 

UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED 

...

OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
NDA
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
OTC MONOGRAPH NOT FINAL
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
BLA
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
NDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
ANDA
AN

OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
UNAPPROVED DRUG OTHER
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
UNAPPROVED HOMEOPATHIC
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
NDA
UNAPPROVED HOMEOPATHIC
UNAPPROVED MEDICAL GAS
BLA
BLA
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOG

OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
ANDA
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
UNAPPROVED HOMEOPATHIC
NDA
OTC MONOGRAPH NOT FINAL
UNAPPROVED DRUG OTHER
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
NDA
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED DRUG OTHER
UNAPPROVED MEDICAL GAS
OTC MONOGRAPH NOT FINAL
ANDA
ANDA
ANDA
ANDA
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
UNAPPROVED HOMEOPATHIC
UNAPPROVED HOMEOPATHIC
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT

OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FI

OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
OTC MONOGRAPH NOT FINAL
NDA
ANDA
ANDA
ANDA
ANDA
NDA
NDA
NDA
NDA
NDA
OTC MONOGRAPH NOT FINAL
NDA
NDA
NDA
NDA
NDA
NDA
NDA
NDA


In [67]:
#Aucune valeur manquante dans package pour la colonne STARTMARKETINGDATE donc on peut utiliser directement product
dateFormatNonValide = 0
for index in range(0,len(product)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(product.iloc[index]['STARTMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
        
print("Nombre de dates de STARTMARKETINGDATE invalides : ",dateFormatNonValide)

Nombre de dates de STARTMARKETINGDATE invalides :  0


In [68]:
endMarketingDateNotnull = product[(~product['ENDMARKETINGDATE'].isnull())]

arrayIndexInvalidEndMarketingDateForProduct = []
dateFormatNonValide = 0
for index, row in endMarketingDateNotnull.iterrows():
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(row['ENDMARKETINGDATE'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        arrayIndexInvalidEndMarketingDateForProduct.append(index)
        dateFormatNonValide +=1
        
print("Nombre de dates de ENDMARKETINGDATE invalide : ",dateFormatNonValide)

Nombre de dates de ENDMARKETINGDATE invalide :  1


In [69]:
#Verifier que la date start soit avant ou égale à la date end (cependant end start peut être non valide car 1 trouvée)
datesIncoherentes = 0
for index in range(0,len(endMarketingDateNotnull)):
    dateEnd = pd.to_datetime(endMarketingDateNotnull.iloc[index]['ENDMARKETINGDATE'])
    dateStart = pd.to_datetime(endMarketingDateNotnull.iloc[index]['STARTMARKETINGDATE'])
    
    if dateEnd < dateStart :
        datesIncoherentes +=1
print("Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide : ",datesIncoherentes)

Nombre de dates incohérentes tel que start soit apres end, ou soit la date start est invalide :  0


In [70]:
listrecordcertifNotNull = product[~product['LISTING_RECORD_CERTIFIED_THROUGH'].isnull()]

dateFormatNonValide = 0
for index in range(0,len(listrecordcertifNotNull)):
    #Si la date n'est pas dans le format indiqué alors cela retourne NaT
    date = pd.to_datetime(listrecordcertifNotNull.iloc[index]['LISTING_RECORD_CERTIFIED_THROUGH'], format='%Y%m%d', errors='coerce')
    if pd.isnull(date):
        dateFormatNonValide +=1
print("Nombre de dates de LISTING_RECORD_CERTIFIED_THROUGH invalides: ", dateFormatNonValide)

Nombre de dates de LISTING_RECORD_CERTIFIED_THROUGH invalides:  0


In [71]:
hh = product[~product['DEASCHEDULE'].isnull()]
print("Differentes possibilites pour DEASCHEDULE :",hh['DEASCHEDULE'].unique())

jj = product[~product['ACTIVE_INGRED_UNIT'].isnull()]
print("Differentes possibilites pour ACTIVE_INGRED_UNIT :",jj['ACTIVE_INGRED_UNIT'].unique())

kk = product[~product['ACTIVE_NUMERATOR_STRENGTH'].isnull()]
erreurActiveNumerator= [isinstance(x, numbers.Number) for x in kk]
print("Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH :",kk['ACTIVE_NUMERATOR_STRENGTH'].unique())
print("Il y a ",len(erreurActiveNumerator), "identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres")

ll = product[~product['ROUTENAME'].isnull()]
print("Nombres de differentes possibilites pour ROUTENAME :",len(ll['ROUTENAME'].unique()))

zz = product[~product['DOSAGEFORMNAME'].isnull()]
print("Nombres de differentes possibilites pour DOSAGEFORMNAME :",len(zz['DOSAGEFORMNAME'].unique()))

cc = product[~product['PROPRIETARYNAMESUFFIX'].isnull()]
nameSuffix = []
for date in cc['PROPRIETARYNAMESUFFIX'].unique():
    if any(char.isdigit() for char in date):
           nameSuffix.append(date)
print("Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX :",len(nameSuffix))

Differentes possibilites pour DEASCHEDULE : ['CV' 'CIV' 'CIII' 'CII']
Differentes possibilites pour ACTIVE_INGRED_UNIT : ['mL/mL' 'mCi/mL' 'mg/.5mL' ... 'g/30mL; g/30mL; g/30mL; 1/30mL' '1/25mL'
 'mg/6g']
Differentes possibilites pour ACTIVE_NUMERATOR_STRENGTH : ['1' '51' '.75' ... '33.2' '415' '20; 25; 33']
Il y a  20 identifiants de ACTIVE_NUMERATOR_STRENGTH qui ne sont pas des nombres
Nombres de differentes possibilites pour ROUTENAME : 180
Nombres de differentes possibilites pour DOSAGEFORMNAME : 134
Nombres de differentes possibilites pour PROPRIETARYNAMESUFFIX : 1455


Pour la colonne <b>DEASCHEDULE</b>, nous ne connaissons pas les valeurs des attributs il est donc difficile de savoir si il y a une incoherence.
<br><br>Pour la colonne <b>PHARM_CLASSES</b>, nous ne pouvons pas evaluer les incoherences en raison des valeurs de la colonne.
<br><br>Pour la colonne <b>ACTIVE_INGRED_UNIT</b>, il y a plusieurs valeurs, cependant en les analysant visuellement, il ne semble pas y avoir d'incoherences.
<br><br>Pour la colonne <b>ACTIVE_NUMERATOR_STRENGTH</b>, il y a des valeurs qui ne sont pas des nombres. Nous ne savons pas si ce sont des incoherences cependant, nous estimons pertinent de le souligner au cas ou cela pourrait poserait probleme plus tard.
<br><br>Pour la colonne <b>SUBSTANCENAME</b> nous ne pouvons pas savoir si les attributs qui sont dans les colonnes sont bons puisque nous n'avons pas de banque permettant de valider. Il en est de meme pour <b>MARKETINGCATEGORYNAME APPLICATIONNUMBER LABELERNAME NONPROPRIETARYNAME PRODUCTID PRODUCTNDC PRODUCTTYPENAME PROPRIETARYNAME</b>, ce sont tous de objets, nous n'avons donc pas beaucoup de details quant a la composition des attributs.
<br><br>Pour la colonne <b>PROPRIETARYNAMESUFFIX</b> nous voyons qu'il y a 1455 elements que leur attribut contient un chiffre dans le suffix. Nous ne pouvons donc pas affirmer que de contenir un chiffre est un incoherence malgre que ce sont des suffix.

#### Conclusion 
Nous avons donc analysé toutes les colonnes afin de voir si il n'y avait pas d'erreurs ou d'incohérence dans les éléments de ces colonnes. Pour les dates nous avons vérifié qu'elles comportaient un mois entre 1 et 12 et un jour entre 1 et 31. Nous ne tenons pas compte de la variance du nombre de jours entre les mois. Lorsque la colonne comportait des objets qui par le nom nous permettait de croire que ce devrait être soi seulement composé de chiffres ou seulement composé de lettres, nous avons vérifié si c'était le cas et compté les nombre d'exceptions afin de vérifier si elles étaient répandues ou pas. Force est de constaté suite à notre analyse qu'il ne semble pas y avoir d'incohérence intra-attributs.

## NOTES 

Donner un sens à nos variables, plus simple pour la relecture

Les incohérences qu'il faudrait qu'on verifie : 

**Package :**

- [x] ProductNDC present dans ProductID
- [x] NDCPackageCode il y a ProductNDC dedans au début 
- [x] SamplePackage E {Y, N}
- [x] NDCExcludeFlag E {Y, N, E, I}
- [x] start et end marketindate ils doivent être cohérent l'un doit être avant l'autre
- [x] verifier format de date valide

**Product :**

- [x] ProductNDC present dans ProductID
- [x] MarkekingCategoryName est au début d'ApplicationNumber A REVOIR NORMALEMENT ON NE PEUT RIEN MONTRER
- [ ] MarkekingCategoryName E {NDA, ANDA, BLA,  OTC Monograph, Unapproved Drug} A REVOIR NORMALEMENT ON NE PEUT RIEN MONTRER
- [x] NDCExcludeFlag E {Y, N, E, I}
- [x] start et end marketindate ils doivent être cohérent l'un doit être avant l'autre
- [x] verifier format de date valide

#### C'est pour la Q6

**Cohérence entre les deux :**

- [ ] Dans les deux tables verifier que le couple ProductID, ProductNDC doit être le même
- [ ] start du package peut pas sortir avant le produit donc doit être avant le start du product  ??? A confirmer mais je pense
- [ ] ndcexcludeflag il faut qu’il soit cohérent entre les deux table, si le produit est exclu, le package aussi 

### Question 3

In [212]:
#### Correction de Package

In [137]:
#Correction de ProductNDC et ProductId 

#Il y a des cas ou il faut transformer la date et le mois en chiffre dans le bon ordre ex) 05-juin->0006-0005, nous en avons corrige 19
correction =""
for ind in arrayIndexIncoherentProductNDCAndProductIdForPackage:
    correction=""
    if str(package.iloc[ind]["PRODUCTNDC"][3:8]).isalpha():
        mois = str(package.iloc[ind]["PRODUCTNDC"][3:8])
        jour = str(package.iloc[ind]["PRODUCTNDC"][0:2])
        if mois == "sept":
            correction+="0009-00"+jour
        elif mois == "juin":
            correction+="0006-00"+jour
        package.loc[ind, "PRODUCTNDC"] = correction #correction dans la table package
        arrayIndexIncoherentProductNDCAndProductIdForPackage.remove(ind)
        
#Les incoherences qu'il reste a corriger(Je n'y vois aucune logique)
display(package.iloc[6913][["PRODUCTID","PRODUCTNDC"]])
for ind in arrayIndexIncoherentProductNDCAndProductIdForPackage:
    display(package.iloc[ind][["PRODUCTID","PRODUCTNDC"]])

PRODUCTID     0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e
PRODUCTNDC                                         0220-2997
Name: 6913, dtype: object

PRODUCTID     25047-762_3becc84b-a4ce-4aff-a7f1-64de3178f3e1
PRODUCTNDC                                          20061122
Name: 36206, dtype: object

PRODUCTID     25047-763_3becc84b-a4ce-4aff-a7f1-64de3178f3e1
PRODUCTNDC                                          20061122
Name: 36207, dtype: object

PRODUCTID     25047-764_3becc84b-a4ce-4aff-a7f1-64de3178f3e1
PRODUCTNDC                                          20061122
Name: 36208, dtype: object

PRODUCTID     25047-765_0d25971c-a335-4603-bb98-ad49bcae8dce
PRODUCTNDC                                          20061122
Name: 36209, dtype: object

...

PRODUCTID     29300-111_4651c75e-b53c-4e27-ad2d-1e8af9111949
PRODUCTNDC                                          20111101
Name: 36701, dtype: object

PRODUCTID     29300-111_4651c75e-b53c-4e27-ad2d-1e8af9111949
PRODUCTNDC                                          20111101
Name: 36702, dtype: object

PRODUCTID     29300-111_4651c75e-b53c-4e27-ad2d-1e8af9111949
PRODUCTNDC                                          20111101
Name: 36703, dtype: object

PRODUCTID     29300-112_4651c75e-b53c-4e27-ad2d-1e8af9111949
PRODUCTNDC                                          20111101
Name: 36704, dtype: object

PRODUCTID     29300-112_4651c75e-b53c-4e27-ad2d-1e8af9111949
PRODUCTNDC                                          20111101
Name: 36705, dtype: object

In [88]:
display(package[["PRODUCTID","PRODUCTNDC"]])

,PRODUCTID,PRODUCTNDC
0,0002-0800_94c48759-29bb-402d-afff-9a713be11f0e,0002-0800
1,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200
2,0002-1200_35551a38-7a8d-43b8-8abd-f6cb7549e932,0002-1200
3,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433
4,0002-1433_42a80046-fd68-4b80-819c-a443b7816edb,0002-1433
...,...,...
173882,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525
173883,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525
173884,99207-525_d47eda34-3952-463c-9597-4225a19dbf13,99207-525
173885,99207-850_76ac026f-e6f1-4d1f-8144-9b5492e2d1bb,99207-850


In [161]:
#Correction des ProductNDC et NDCPackageCode, nous avons ajuster les erreurs du type 05-juin->0006-0005. Il y en avait 19
#Cette boucle vient enlever les index corriger de ProductNDC a la case precedente

for ind in arrayIndexIncoherentProductNDCAndNDCPackageCode:
    if package.iloc[ind]["PRODUCTNDC"] == package.iloc[ind]["NDCPACKAGECODE"][0:9]:
        arrayIndexIncoherentProductNDCAndNDCPackageCode.remove(ind)

#Nous avons le cas interessant ici ou semble y avoir une erreur dasn NDCPACKAGECODE qui entraine une erreur dasn ProductID
#PRODUCTID         0220-2997_8590fcc9-c0c3-f43b-e053-2991aa0a692e
#PRODUCTNDC                                             0220-2997
#NDCPACKAGECODE                                      0220-1997-41
#Name: 6913, dtype: object
#correction de cette erreur dans les 3 attributs
code = package.iloc[6913]["NDCPACKAGECODE"][:9]
package.loc[6913,"PRODUCTNDC"]= code
new = code+package.iloc[6913]["PRODUCTID"][9:]
package.loc[6913,"PRODUCTID"]= new

PRODUCTID         25047-762_3becc84b-a4ce-4aff-a7f1-64de3178f3e1
PRODUCTNDC                                              20061122
NDCPACKAGECODE                                      25047-762-44
Name: 36206, dtype: object

36206

PRODUCTID         25047-763_3becc84b-a4ce-4aff-a7f1-64de3178f3e1
PRODUCTNDC                                              20061122
NDCPACKAGECODE                                      25047-763-44
Name: 36207, dtype: object

36207

PRODUCTID         25047-764_3becc84b-a4ce-4aff-a7f1-64de3178f3e1
PRODUCTNDC                                              20061122
NDCPACKAGECODE                                      25047-764-44
Name: 36208, dtype: object

...

41914

PRODUCTID         37808-018_75455e7f-2868-4a65-a91d-5ab4d21e6fa6
PRODUCTNDC                                             37808-018
NDCPACKAGECODE                                          20201130
Name: 41928, dtype: object

41928

PRODUCTID         37808-038_6b93328d-58b5-49b8-aa23-c3954b44a6c6
PRODUCTNDC                                             37808-038
NDCPACKAGECODE                                          20201231
Name: 41946, dtype: object

41946

In [210]:
#Toute les erreurs sont la meme, l'annee est ecrit 2997 au lieu de 1997, nous corrigeons 243 erreurs
for date in arrayIndexInvalidStartMktgDateForPackage:
    nDate = "1"+str(package.iloc[date]["STARTMARKETINGDATE"])[1:]
    package.loc[date,"STARTMARKETINGDATE"]=nDate

In [181]:
for ind in arrayIndexIncoherentProductIDAndNDCPackageCode:
    display(package.iloc[ind][["PRODUCTID", "PRODUCTNDC","NDCPACKAGECODE"]])

PRODUCTID         37000-005_99fe4eef-411e-08db-e053-2995a90a295e
PRODUCTNDC                                             37000-005
NDCPACKAGECODE                                          20211201
Name: 40965, dtype: object

PRODUCTID         37000-005_99fe4eef-411e-08db-e053-2995a90a295e
PRODUCTNDC                                             37000-005
NDCPACKAGECODE                                          20211201
Name: 40966, dtype: object

PRODUCTID         37000-005_99fe4eef-411e-08db-e053-2995a90a295e
PRODUCTNDC                                             37000-005
NDCPACKAGECODE                                          20211201
Name: 40967, dtype: object

PRODUCTID         37000-005_99fe4eef-411e-08db-e053-2995a90a295e
PRODUCTNDC                                             37000-005
NDCPACKAGECODE                                          20211201
Name: 40971, dtype: object

PRODUCTID         37000-005_99fe4eef-411e-08db-e053-2995a90a295e
PRODUCTNDC                                             37000-005
NDCPACKAGECODE                                          20211201
Name: 40972, dtype: object

...

PRODUCTID         37205-941_99993239-566c-4203-afc1-22c7438d4155
PRODUCTNDC                                             37205-941
NDCPACKAGECODE                                          20200531
Name: 41910, dtype: object

PRODUCTID         37205-980_80df59b0-86a6-4493-aec4-f418223a5c3c
PRODUCTNDC                                             37205-980
NDCPACKAGECODE                                          20200831
Name: 41913, dtype: object

PRODUCTID         37205-980_80df59b0-86a6-4493-aec4-f418223a5c3c
PRODUCTNDC                                             37205-980
NDCPACKAGECODE                                          20200831
Name: 41914, dtype: object

PRODUCTID         37808-018_75455e7f-2868-4a65-a91d-5ab4d21e6fa6
PRODUCTNDC                                             37808-018
NDCPACKAGECODE                                          20201130
Name: 41928, dtype: object

PRODUCTID         37808-038_6b93328d-58b5-49b8-aa23-c3954b44a6c6
PRODUCTNDC                                             37808-038
NDCPACKAGECODE                                          20201231
Name: 41946, dtype: object